In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['contractions', 'glove840b300dtxt', 'swear-list', 'jigsaw-unintended-bias-in-toxicity-classification', 'fasttext-crawl-300d-2m', 'cleaned']


In [2]:
import numpy as np
import pandas as pd
import gc
import re
from keras.preprocessing import text, sequence
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Embedding, SpatialDropout1D, add, concatenate
from keras.layers import CuDNNLSTM, CuDNNGRU, GlobalAveragePooling1D, GlobalMaxPooling1D, Bidirectional
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from sklearn.externals import joblib
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from joblib import Parallel, delayed

pd.set_option('display.max_colwidth', 300)

Using TensorFlow backend.


Pre trained files

In [3]:
EMBEDDING_FILES = [
    '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec',
    '../input/glove840b300dtxt/glove.840B.300d.txt']

In [4]:
NUM_MODELS = 2 
BATCH_SIZE = 1024
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4*LSTM_UNITS #??
EPOCHS = 4
MAX_LEN = 220

In [5]:
IDENTITY_COLUMNS = [
    'transgender', 'female', 'homosexual_gay_or_lesbian', 'muslim', 'hindu',
    'white', 'black', 'psychiatric_or_mental_illness', 'jewish']  

In [ ]:
AUX_COLUMNS = ['target', 'severe_toxicity','obscene','identity_attack','insult','threat']
TEXT_COLUMN = 'comment_text'
TARGET_COLUMN = 'target'
CHARS_TO_REMOVE = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

In [6]:
contractions = joblib.load('../input/contractions/contraction_mapping')

In [7]:
swear_words = pd.read_table('../input/swear-list/swear_wordlist.txt')
swear_words.columns = ['words']
swear_words['words'] = swear_words['words'].map(lambda x: x.strip())
swear_words = swear_words['words'].tolist()

In [ ]:
replace_with_fuck = []

for swear in swear_words:
    replace_with_fuck.append(swear)
        
replace_with_fuck = '|'.join(replace_with_fuck)

In [8]:
def handle_swears(text):
    text = re.sub(replace_with_fuck, ' fuck ', text)
    return text

def clean_sentence(text, mapping):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    
    text = ' '.join([mapping[t] if t in mapping else t for t in text.split(" ")])
    
    text = handle_swears(text)
        
    wordnet_lemmatizer = WordNetLemmatizer()
    stop = set(stopwords.words('english'))
    
    VERB_CODES = {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', }
    ADJ_CODES = {'JJ','JJR','JJS'}
    ADV_CODES = {'RB','RBR','RBS'}
    NOUN_CODES = {'NN', 'NNP','NNS','NNPS'}
    
    new_sentence = []
    #words = nltk.word_tokenize(text)
    #role = nltk.pos_tag(words)
    
    for i, word in enumerate(words):
        if role[i][1] in VERB_CODES: 
            new_word = wordnet_lemmatizer.lemmatize(word, 'v')
        else:
            if role[i][1] in ADJ_CODES: 
                new_word = wordnet_lemmatizer.lemmatize(word, 'a')
            else:
                if role[i][1] in ADV_CODES: 
                    new_word = wordnet_lemmatizer.lemmatize(word, 'r')
                else:
                    if role[i][1] in NOUN_CODES: 
                        new_word = wordnet_lemmatizer.lemmatize(word, 'n') 
                    else:
                        new_word = wordnet_lemmatizer.lemmatize(word)
            
        if new_word not in stop and new_word.isalpha() and len(new_word)>1:
            new_sentence.append(new_word)
        
    s = ' '.join(new_sentence)
   
    return s
    
def get_coefs(word, *arr):
    """
    Get word, word_embedding
    """
    return word, np.asarray(arr,dtype='float32')


In [9]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in f)

In [10]:
def build_matrix(word_index, path):
    """
    Here we take each word we've tokenized in our text corpus
    for each word we look up in the pre-trained embedding.
    Each row in this matrix is a corpus word's embedding.
    """
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index)+1, 300))
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            pass
    return embedding_matrix

In [ ]:
def build_model(embedding_matrix, num_aux_targets):#, loss_weight):
    """
    embedding layer
    droput layer
    2 * bidirectional LSTM layers
    2 * pooling layers
    2 dense layers
    1 softmax layer
    """
    words = Input(shape=(MAX_LEN,)) 
    #Embedding layer takes variable size input
    x = Embedding(*embedding_matrix.shape, weights = [embedding_matrix], trainable=False)(words)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    x = Bidirectional(CuDNNLSTM(LSTM_UNITS, return_sequences=True))(x)
    
    hidden = concatenate([ 
        GlobalMaxPooling1D()(x),
        GlobalAveragePooling1D()(x)
        ])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    hidden = add([hidden, Dense(DENSE_HIDDEN_UNITS, activation='relu')(hidden)])
    result = Dense(1, activation='sigmoid')(hidden)
    aux_result =Dense(num_aux_targets, activation='sigmoid')(hidden)

    model = Model(inputs =words, outputs =[result, aux_result])
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    return model

In [11]:
train_df = pd.read_csv('../input/cleaned/cleaned_train_df.csv')
test_df = pd.read_csv('../input/cleaned/cleaned_test_df.csv')

In [12]:
train_df

,Unnamed: 0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,bisexual,black,buddhist,christian,female,heterosexual,hindu,homosexual_gay_or_lesbian,intellectual_or_learning_disability,jewish,latino,male,muslim,other_disability,other_gender,other_race_or_ethnicity,other_religion,other_sexual_orientation,physical_disability,psychiatric_or_mental_illness,transgender,white,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,0,59848,0.000000,This cool It like want mother read Really great idea well,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:41.987077+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
1,1,59849,0.000000,Thank This would make life lot less Keep let anyone get way,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:42.870083+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
2,2,59852,0.000000,This urgent design problem kudos take Very impressive,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:45.222647+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
3,3,59855,0.000000,Is something able install site When release,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:47.601894+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
4,4,59856,0.893617,haha guy bunch loser,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2015-09-29 10:50:48.488476+00,2,NaN,2006,rejected,0,0,0,1,0,0.000000,4,47
5,5,59859,0.666667,ur comment,0.047619,0.638095,0.000000,0.333333,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:50.865549+00,2,NaN,2006,rejected,0,0,0,0,0,0.009524,0,105
6,6,59861,0.457627,hahahahahahahahhha suck,0.050847,0.305085,0.000000,0.254237,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:52.451277+00,2,NaN,2006,rejected,0,0,0,0,0,0.220339,0,59
7,7,59863,0.000000,FFFFUUUUUUUUUUUUUUU,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-09-29 10:50:54.055221+00,2,NaN,2006,rejected,0,0,0,0,0,0.000000,0,4
8,8,239575,0.000000,The rancher seem motivate mostly greed one right allow animal destroy public land,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-13 18:01:05.156229+00,6,NaN,26662,approved,0,0,0,0,0,0.000000,0,4
9,9,239576,0.000000,It great show Not combo would expect good together,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-13 18:11:21.570460+00,6,239522.0,26650,approved,0,0,0,1,0,0.000000,0,4


In [ ]:

#train_df['comment_text'] = train_df['comment_text'].apply(lambda x: clean_sentence(x, contractions))
#test_df['comment_text'] = test_df['comment_text'].apply(lambda x: clean_sentence(x, contractions))


In [ ]:
#train_tokens = Parallel(n_jobs=-1, verbose=1)(delayed(clean_sentence)(text,contractions) for text in train_df['comment_text'].tolist())

In [ ]:
#test_tokens = Parallel(n_jobs=-1, verbose=1)(delayed(clean_sentence)(text,contractions) for text in test_df['comment_text'].tolist())

In [ ]:
#train_df['comment_text'] = train_tokens
#test_df['comment_text'] = test_tokens


In [ ]:
#del train_tokens, test_tokens
gc.collect()
gc.collect()

In [ ]:
x_train = train_df[TEXT_COLUMN].astype(str)
y_train = train_df[TARGET_COLUMN].values
y_aux_train = train_df[AUX_COLUMNS].values
x_test = test_df[TEXT_COLUMN].astype(str)

In [ ]:
for column in IDENTITY_COLUMNS + [TARGET_COLUMN]:
    train_df[column] = np.where(train_df[column] >=0.5, True, False)

In [ ]:
#Return a tokenizer class
tokenizer = text.Tokenizer(filters = CHARS_TO_REMOVE)
tokenizer.fit_on_texts(list(x_train)+ list(x_test))
# Turn text to sequences of tokens
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [ ]:
#Pad sequences to the same length
x_train = sequence.pad_sequences(x_train,maxlen=MAX_LEN)
x_test= sequence.pad_sequences(x_test, maxlen=MAX_LEN)


In [ ]:
# Initialize weights
sample_weights = np.ones(len(train_df), dtype=np.float32)
# Add all the values of the identities along rows
sample_weights += train_df[IDENTITY_COLUMNS].sum(axis=1)
#Add all values of targets*~identity
sample_weights += train_df[TARGET_COLUMN]*(~train_df[IDENTITY_COLUMNS]).sum(axis=1)
#Add all values ~targets*identity
sample_weights += (~train_df[TARGET_COLUMN])*train_df[IDENTITY_COLUMNS].sum(axis=1)
#Normalize them
sample_weights/=sample_weights.mean()

In [ ]:
embedding_matrix = np.concatenate([build_matrix(tokenizer.word_index,f) for f in EMBEDDING_FILES], axis =-1)
del train_df, tokenizer
gc.collect()

checkpoint_predictions = []
weights = []

In [ ]:
for model_idx in range(NUM_MODELS):
    #Passes embedding matrix and aux outputs shape
    model = build_model(embedding_matrix, y_aux_train.shape[-1]) #1/sample_weights.mean())
    for global_epoch in range(EPOCHS):
        model.fit(
            x_train,
            [y_train, y_aux_train],
            batch_size=BATCH_SIZE,
            epochs=1,
            verbose=2,
            sample_weight=[sample_weights.values, np.ones_like(sample_weights)],
            callbacks = [
                LearningRateScheduler(lambda _: 1e-3*(0.55**global_epoch))
                ]
        )
    
        checkpoint_predictions.append(model.predict(x_test, batch_size=BATCH_SIZE)[0].flatten())
        weights.append(2 ** global_epoch)
    del model
    gc.collect()

In [ ]:
gc.collect()

In [ ]:
len(checkpoint_predictions)

In [ ]:
predictions = np.average(checkpoint_predictions, weights=weights, axis=0)
submission = pd.DataFrame.from_dict({
        'id': test_df.id,
        'prediction': predictions})
submission.to_csv('submission.csv', index=False)